In [1]:
push!(LOAD_PATH,"../src/")
using Word2Vec
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:6
in gc_protect_handle at 
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:6
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:6
in gc_protect_handle at /home/ubuntu/.julia/v0.5/ZMQ/src/ZMQ.jl
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:6
in gc_protect_handle at /home/ubuntu/.julia/v0.5/ZMQ/src/ZMQ.jl
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:6
in gc_protect_handle at /home/ubuntu/.julia/v0.5/ZMQ/src/ZMQ.jl

Use "(::Type{NullableArray{T,N}})(...)" instead.

Use "(::Base.MinFun)(...)" instead.

Use "(::Base.MaxFun)(...)" instead.


@pz (macro with 1 method)

In [2]:
ee = restore("models/text8_v1.model")

Word embedding(dimension = 30)of 71290 words, trained on 83594170 words


In [33]:
function prob_of_context{S<:AbstractString}(embed::Word2Vec.WordEmbedding, context::Vector{S},middle::Int)
    prob_of_context(embed, context[1:middle-1], context[middle], context[middle-1:end])
end
    
function prob_of_context{S<:AbstractString}(embed::Word2Vec.WordEmbedding, lwords::Vector{S},middle_word::S, rwords::Vector{S})
    
    context_words = [lwords;rwords]

    total_logprob=0.0 #Work in logprob to avoid underflow, and get more stability
    for target_word in context_words
        # discard words not presenting in the classification tree
        (haskey(embed.codebook, target_word) && haskey(embed.codebook, middle_word)) || continue
        node = embed.classification_tree      
        input = embed.embedding[middle_word]
        
        word_logprob = 0.0
        for code in embed.codebook[target_word]  
            word_logprob+=log(Word2Vec.predict(node.data, input)[code])
            node = node.children[code]
        end
        total_logprob+=word_logprob
    end
    exp(total_logprob) #Going back out of the log domain is not required for external logic, but it is nice for clarity
    
end

prob_of_context (generic function with 2 methods)

In [34]:
prob_of_context(ee,split("the british queen is the ruler of england scottland and wales"), 6)

7.512724083027164e-26

In [35]:
prob_of_context(ee,split("the american queen is the ruler of england scottland and wales"), 6)

2.3027070396138277e-26

In [36]:
for ii in 2:8
    println(prob_of_context(ee,split("the american queen is not the ruler of england scottland and wales"), ii))
end

3.3352301203759964e-32
2.761047836716399e-29
6.917343841649315e-35
7.747502127414158e-34
1.7170909184966057e-32
2.8220310125745205e-29
1.7167975861431585e-32


In [37]:
prob_of_context(ee,split("hello no hello hello hello hello"), 3)

1.2893581516517601e-23

In [23]:
log(1)

0.0